#### Testing for colllapsing and expanding trees of particles

In [1]:
import numpy aclass Beta(object):
    
    def __init__(self):
        self.weight = 1.0
        

class Alpha(object):
    
    def __init__(self):
        
        self.weight = 0.0
        self.beta_alpha_tree = None
    
    def make_beta_tree(self, beta_num):
        self.beta_alpha_tree = [Beta() for particle in range(beta_num)]
        

class Root(object):
    
    def __init__(self, alpha_num, beta_num):
        
        self.alpha_list = [Alpha() for particle in range(alpha_num)]
        self.alpha_num = alpha_num
        self.beta_num = beta_num
        
    def get_weights(self):
        
        posteriorweights = []
        
        counter_a = 0.0
        for alpha_particle in self.alpha_list:
            
            alpha_particle.weight = (1.0 / self.alpha_num) *counter_a
            alpha_particle.make_beta_tree(self.beta_num) # makes beta tree
            
            counter_b=0.0
            for beta_particle in alpha_particle.beta_alpha_tree:
                beta_particle.weight = (1.0 / self.beta_num) * counter_b
                
                joint_weight = beta_particle.weight * alpha_particle.weight
                
                posteriorweights.append([joint_weight])
                counter_b +=1.0
            
            counter_a += 1.0
        return posteriorweights
    
    def set_new_particles(self, resampled_indices):
        
        new_particles = []
        for idx in resampled_indices:
            
            alpha_index = int(idx//float(self.beta_num))
            beta_alpha_index = int(idx - alpha_index*self.beta_num)
            
            add_new_particle = self.alpha_list[alpha_index].beta_alpha_tree[beta_alpha_index]
            
            new_particles.append(add_new_particle)
        
        return new_particles
        
        
        

root = Root(3,4)
posterior_weights = np.asarray(root.get_weights()).flatten()
reshaped_weights = posterior_weights.reshape(root.alpha_num,root.beta_num) 
resampled_indices = np.floor(np.random.uniform(low=0, high=11, size=12))
new_part = root.set_new_particles(resampled_indices)

In [112]:
"HEY"

'HEY'

In [2]:
posterior_weights

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.08333333,  0.16666667,  0.25      ,  0.        ,  0.16666667,
        0.33333333,  0.5       ])

In [3]:
resampled_indices

array([  7.,   4.,   6.,  10.,   1.,   3.,   0.,   2.,   9.,   4.,   0.,
         7.])

In [104]:
n = 3
print n
n *= 1.5
print n

3
4.5


In [111]:
sufficient_sample=False
num_of_samples=1
while sufficient_sample is False:
    num_of_samples *=2 # exponentially growing search 
    resample_index = np.random.uniform(low=0, high=10, size=num_of_samples)
    unique_alphas = list(set([get_alpha_node_from_treeleaf(leafy, pset_beta) for leafy in resample_index]))
    print unique_alphas
    if len(unique_alphas) == pset_alpha:
        sufficient_sample = True

[1]
[0, 1]
[0, 1, 2]


In [6]:
weights = [nx.weight for nx in new_part]
print weights

[0.75, 0.0, 0.5, 0.5, 0.25, 0.75, 0.0, 0.5, 0.25, 0.0, 0.0, 0.75]


In [97]:
def get_subtrees(self, resampled_indices):
    '''docstring'''
    
    new_sub_trees = []
    
    resampled_indices.sort()
    alpha_index_0 = None
    strt_counter=0
    end_counter=0

    for idx in resampled_indices:

        alpha_index = get_alpha_node_from_treeleaf(idx, self.beta_num)
        beta_alpha_idx = get_beta_node_from_treeleaf(idx, self.beta_num)

        if alpha_index_0 == alpha_index:
            end_counter +=1
            
        elif alpha_index_0 != alpha_index:

            new_sub_trees.append([strt_counter, end_counter])
            
            alpha_index_0 = alpha_index
            strt_counter = end_counter
            end_counter += 1
    
    if end_counter == len(resampled_indices):
        end_counter += 1
        new_sub_trees.append([strt_counter, end_counter])
    
    return new_sub_trees
            

def collapse_beta(self, new_sub_trees, resampled_indices, pset_beta):
    
    state_update = 0.
    
    new_alpha_particle_list=[]
    for pairs in new_sub_trees:
        
        subtree = resampled_indices[pairs[0]:pairs[1]]
        leaf_count = float(len(subtree))
        normaliser = (1./leaf_count)
        
        alpha_node = get_alpha_node_from_treeleaf(pairs[0], pset_beta)
        beta_alpha_nodes = [get_beta_node_from_treeleaf(leafy, pset_beta) for leafy in subtree]
        
        for node in beta_alpha_nodes:
            beta_state = self.AlphaSet[alpha_node].Beta_Alpha_j[node].particle
            beta_lengthscale = beta_state[self.AlphaSet[alpha_node].node_j]
            len_scale_update += normaliser*beta_lengthscale
            
        parent = self.AlphaSet[alpha_node].particle 
        parent[self.AlphaSet[alpha_node].node_j] = len_scale_update
        
        # Beta Layer Collapsed
        self.AlphaSet[alpha_node].particle = parent
        self.AlphaSet[alpha_node].BetaAlphaSet_j = None
    
        # New Alphas Stored
        new_alpha_particle.append(self.AlphaSet[alpha_node])
    
    return new_alpha_particle_list
            
        
@staticmethod
def get_alpha_node_from_treeleaf(leaf_index, pset_beta):
    alpha_node = int(leaf_index//float(pset_beta))
    return alpha_node

@staticmethod
def get_beta_node_from_treeleaf(leaf_index, pset_beta):
    beta_node = int(leaf_index - int(leaf_index//float(pset_beta))*pset_beta)
    return beta_node

In [98]:
subtrees = get_subtrees(resampled_indices)
print subtrees
for idx in subtrees:
    print idx

Under the same alpha, we get:  (0, 0)
Under the same alpha, we get:  (0, 0)
Under the same alpha, we get:  (0, 1)
Under the same alpha, we get:  (0, 2)
Under the same alpha, we get:  (0, 3)
NEW alpha pair, we get:  (1, 0)
Under the same alpha, we get:  (1, 0)
Under the same alpha, we get:  (1, 2)
Under the same alpha, we get:  (1, 3)
Under the same alpha, we get:  (1, 3)
NEW alpha pair, we get:  (2, 1)
Under the same alpha, we get:  (2, 2)
calc last pair
[[0, 5], [5, 10], [10, 13]]
[0, 5]
[5, 10]
[10, 13]


In [99]:
collapse_beta(subtrees, resampled_indices)

alpha subtree 0
beta leaves [0, 0, 1, 2, 3]
alpha subtree 1
beta leaves [0, 0, 2, 3, 3]
alpha subtree 2
beta leaves [1, 2]


In [79]:
print subtrees
print
print

for pairs in subtrees:
    print "pair", pairs
    print
    print "slice for the pair", 
    print
    print

 [[0, 5], [5, 10], [10, 13]]


pair [0, 5]

slice for the pair [ 0.  0.  1.  2.  3.]


pair [5, 10]

slice for the pair [ 4.  4.  6.  7.  7.]


pair [10, 13]

slice for the pair [  9.  10.]




In [53]:
collapse_beta(resampled_indices)

Under the same alpha, we get:  (0, 0)
Under the same alpha, we get:  (0, 0)
Under the same alpha, we get:  (0, 1)
Under the same alpha, we get:  (0, 2)
Under the same alpha, we get:  (0, 3)
NEW alpha pair, we get:  (1, 0)
Under the same alpha, we get:  (1, 0)
Under the same alpha, we get:  (1, 2)
Under the same alpha, we get:  (1, 3)
Under the same alpha, we get:  (1, 3)
NEW alpha pair, we get:  (2, 1)
Under the same alpha, we get:  (2, 2)


[[(0, 6)], [(3, 11)]]

In [19]:
collapse_beta(resampled_indices)

Under the same alpha, we get:  (0, 0)
Under the same alpha, we get:  (0, 0)
Under the same alpha, we get:  (0, 1)
Under the same alpha, we get:  (0, 2)
Under the same alpha, we get:  (0, 3)
NEW alpha pair, we get:  (1, 0)
Under the same alpha, we get:  (1, 0)
Under the same alpha, we get:  (1, 2)
Under the same alpha, we get:  (1, 3)
Under the same alpha, we get:  (1, 3)
NEW alpha pair, we get:  (2, 1)
Under the same alpha, we get:  (2, 2)


####  >> Extracting alpha, beta particles that have been resampled

In [10]:
'''
each resampled index represents an (alpha, beta) particle
a repeated index means that the same (alpha, beta) particle is repeated

we want to 

--> go to each tree
--> delete beta particles under every alpha
--> add beta particles if they are resampled more than once
--> take the posterior lenthscale for each alpha
---> replace the posterior lengthscale for that alpha
---> replace all the alpha particles with resampled values
---> how do we conserve alpha particle number?


each row in reshaped indices is a alpha particle, elemtns of the row are beta leaves in the original tree

'''

strt = 0
end = 0
resampled_indices.sort()

for idx in range(len(resampled_indices)):

    if resampled_indices[idx] == resampled_indices[idx-1]:
            end += 1

    elif resampled_indices[idx] != resampled_indices[idx-1]:
        print "the following is a start end pair:", (strt, end)
        print "indices extracted", resampled_indices[strt: end]
        print "resetting pairs..."
        strt = idx
        end = idx +1
        print
        print

    if idx == len(resampled_indices)-1:
        end = idx + 1
        print "the last pair is", (strt, end)
        print "indices extracted", resampled_indices[strt: end]

the following is a start end pair: (0, 0)
indices extracted []
resetting pairs...


the following is a start end pair: (0, 2)
indices extracted [ 0.  0.]
resetting pairs...


the following is a start end pair: (2, 3)
indices extracted [ 1.]
resetting pairs...


the following is a start end pair: (3, 4)
indices extracted [ 2.]
resetting pairs...


the following is a start end pair: (4, 5)
indices extracted [ 3.]
resetting pairs...


the following is a start end pair: (5, 7)
indices extracted [ 4.  4.]
resetting pairs...


the following is a start end pair: (7, 8)
indices extracted [ 6.]
resetting pairs...


the following is a start end pair: (8, 10)
indices extracted [ 7.  7.]
resetting pairs...


the following is a start end pair: (10, 11)
indices extracted [ 9.]
resetting pairs...


the last pair is (11, 12)
indices extracted [ 10.]


#### Testing the index extraction for alpha, beta coordinates from a flattened ordered array

In [ ]:
import numpy as np 
posterior_weights = np.asarray(root.get_weights()).flatten()
reshaped_weights = posterior_weights.reshape(root.alpha_num,root.beta_num) # view of the original array
print reshaped_weights

In [ ]:
posterior_weights[-1] = 0.5
print posterior_weights

In [ ]:
indices = np.arange(root.alpha_num * root.beta_num)
print indices.reshape(root.alpha_num,root.beta_num)

In [ ]:
resampled_indices = np.floor(np.random.uniform(low=0, high=11, size=12))

In [ ]:
print "for every alpha, there are %s beta particles" %(root.beta_num)
print 
print

for idx in resampled_indices:
    print "the index is", idx
    dev = idx//float(root.beta_num)
    print "Integer division is", dev
    print "Did we select the row containing indices?" , indices.reshape(root.alpha_num,root.beta_num)[dev, :]
    col = idx - dev*root.beta_num
    print "predict col index as", col
    print "recover the resampled index", indices.reshape(root.alpha_num,root.beta_num)[dev, int(col)]
    print
    

#### Testing for copy and view effects when using assignments for class atrtributes and properties

In [ ]:
class AttributeClass(object):
    
    def __init__(self):
        
        self.__some_attribute = 0.0
        
    @property
    def some_attribute(self):
        return self.__some_attribute
    @some_attribute.setter
    def some_attribute(self, new_attr_val):
        self.__some_attribute = new_attr_val
    

In [ ]:
test_attribute = AttributeClass()
print test_attribute
print test_attribute.some_attribute
type(test_attribute.some_attribute)

In [ ]:
b = test_attribute.some_attribute # assign to b
print "orginal b =", b # check b
b = 3.0 # assign some other value to b
print "new b=", b # check b
print "orginal attribute=", test_attribute.some_attribute # check attribute

In [ ]:
b = test_attribute.some_attribute # assign to b
print "orginal b =", b # check b
test_attribute.some_attribute = 3.0 # assign some other value to attribute
print "new attribute=", test_attribute.some_attribute # check attribute
print " (should be orginal) b=", b # check b

#### Testing how to pass dictionaries of dictionaries and functions into class arguments

In [ ]:
class Test:
    
    def __init__(self, **DICT):
        
        self.dict = DICT
        pass

    
    def calc_weights_set(self):
        
        self.dict["function"](0, **self.dict["args"])

In [ ]:
def cat_func(num, **args):
    print num
    for key in args.keys():
        print (key, args[key])

In [ ]:
inputdict = {"function": cat_func, "args": {"shifter": "ok", "uli":"great"} }

In [ ]:
testobj = Test(**inputdict)

In [ ]:
print testobj.dict.keys()
print testobj.dict["args"].values()

In [ ]:
type(testobj.dict["args"].values())

In [ ]:
testobj.calc_weights_set()

#### Testing for functions to calculate the maximal pairwise distance from a set of values

In [ ]:
list_pos = [(0.0, 0.0), (1.0, 0.0), (2.0, 0.0), (5.0, 3.0)]

In [ ]:
import numpy as np

def compute_dist(one_pair):
    x, y = one_pair
    return np.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2 )

def find_max_distance(list_of_positions):
    distance_pairs = [a_pair for a_pair in combinations(list_of_positions,2)]
    distances = [compute_dist(one_pair)for one_pair in distance_pairs]
    return max(distances), distance_pairs[np.argmax(distances)]

